# Подготовка данных по недвижимости. Релиз 2

Задача этапа: закоммитить результаты очистки данных от пропусков.

состав репозитория дополняется:
preprocessing.ipynb - jupyter notebook файл, где показан процесс обработки данных
data.csv - отвечающий критериям:
названия колонок на английском языке в одно/несколько слов с нижним подчёркиванием
в каждой колонке должны отсутствовать пропущенные значения (NaN, None и т.д.)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

Рассмотрим датасет

In [2]:
df = pd.read_csv('/home/user/ds_bootcamp/real_estate/_data.csv')
df.head()

,Unnamed: 0,ID объявления,Количество комнат,Тип,Метро,Адрес,"Площадь, м2",Дом,Парковка,Цена,...,Окна,Санузел,Можно с детьми/животными,Дополнительно,Название ЖК,Серия дома,"Высота потолков, м",Лифт,Мусоропровод,Ссылка на объявление
0,0,271271157,4,Квартира,м. Смоленская (9 мин пешком),"Москва, улица Новый Арбат, 27",200.0/20.0,"5/16, Монолитный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,NaN,NaN,"Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Ванна, Душ...","Новый Арбат, 2010",NaN,3.0,"Пасс (4), Груз (1)",Да,https://www.cian.ru/rent/flat/271271157
1,1,271634126,4,Квартира,м. Смоленская (8 мин пешком),"Москва, улица Новый Арбат, 27",198.0/95.0/18.0,"5/16, Монолитно-кирпичный",подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,"Совмещенный (2), Раздельный (1)",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.5,"Пасс (1), Груз (1)",Нет,https://www.cian.ru/rent/flat/271634126
2,2,271173086,"4, Оба варианта",Квартира,м. Смоленская (7 мин пешком),"Москва, улица Новый Арбат, 27",200.0/116.0/4.0,5/16,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",...,На улицу и двор,Совмещенный (3),Можно с детьми,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",Новый Арбат,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/271173086
3,3,272197456,"4, Оба варианта",Квартира,м. Смоленская (3 мин пешком),"Москва, переулок Плотников, 21С1",170.0/95.0/17.0,5/6,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",...,На улицу и двор,Совмещенный (3),Можно с животными,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",NaN,NaN,3.2,Пасс (1),NaN,https://www.cian.ru/rent/flat/272197456
4,4,273614615,2,Квартира,м. Арбатская (7 мин пешком),"Москва, улица Новый Арбат, 15",58.0/38.0/5.0,"12/26, Панельный",NaN,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",...,На улицу и двор,Совмещенный (2),NaN,"Мебель в комнатах, Мебель на кухне, Ванна, Душ...",The Book,NaN,3.9,"Пасс (1), Груз (1)",Да,https://www.cian.ru/rent/flat/273614615


In [3]:
df.columns

Index(['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро',
       'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны',
       'Описание', 'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел',
       'Можно с детьми/животными', 'Дополнительно', 'Название ЖК',
       'Серия дома', 'Высота потолков, м', 'Лифт', 'Мусоропровод',
       'Ссылка на объявление'],
      dtype='object')

In [4]:
df = df[
       ['Unnamed: 0', 'ID  объявления', 'Количество комнат', 'Тип', 'Метро',
       'Адрес', 'Площадь, м2', 'Дом', 'Парковка', 'Цена', 'Телефоны',
       'Описание', 'Ремонт', 'Площадь комнат, м2']
     ] # Делим таблицу на две части для работы по столбцам

## Обрабатываем данные по признакам

### Unnamed: 0

Удаляем столбец.

In [5]:
df = df.drop('Unnamed: 0', axis=1)

### Комнаты

Комнаты - параметр, значительно влияющий на цену. Выводить значения в пропусках некорректно. Удалим пропуски 

In [6]:
df.dropna(subset=['Количество комнат'], inplace=True)

Преобразуем значения к числовому виду

In [7]:
df['Количество комнат'] = df['Количество комнат'].map(lambda x: int(x.split(', ')[0]))

### Тип

Удаляем признак, т.к. одинаковые значения 'Квартира'

In [8]:
df = df.drop('Тип', axis=1)

Поскольку один тип значений - он не повлияет на модель, но может усложнить расчет, признак к удалению.

### Метро

Удаляем строки с пропусками по значению метро. Заменить не сможем.

In [9]:
df.dropna(subset=['Метро'], inplace=True)

In [10]:
df['Метро'].unique()

array(['м. Смоленская (9 мин пешком)', 'м. Смоленская (8 мин пешком)',
       'м. Смоленская (7 мин пешком)', ..., 'м. Солнцево (16 мин пешком)',
       'м. Боровское шоссе (5 мин на машине)',
       'м. Солнцево (5 мин на машине)'], dtype=object)

Выделим отдельно 'метро': 'Смоленская'

In [11]:
df['метро'] = df['Метро'].map(lambda x: x.split()[1])

In [12]:
df = df[df['Метро'].map(lambda x: x[x.index('('):].split()[0][1:]) != 'None'] # удалим строки, в которых None по времени до метро
df['Время до метро'] = df['Метро'].map(lambda x: x[x.index('('):].split()[0][1:]) # время до метро влияет на цену

In [13]:
df['Расчет времени пешком'] = df['Метро'].map(lambda x: x.split()[-1][-7:-1]=='пешком') #важно, как посчитано время до метро

In [14]:
df = df.drop('Метро', axis=1) #удаляем исходный столбец про метро

In [15]:
len(df) #проверяем, сколько мы отсесяли строк. отсесяли 3 тысячи из 23. Считаем это необходимым для качественной работы модели.

20193

### Адрес

In [16]:
df['Адрес'].unique() # Рассмотрим вид адреса

array(['Москва, улица Новый Арбат, 27',
       'Москва, переулок Плотников, 21С1',
       'Москва, улица Новый Арбат, 15', ...,
       'Москва, Боровский проезд, 11',
       'Москва, улица Богданова, 6к1, ш. Боровское (3 км до МКАД), ш. Сколковское (8 км до МКАД)',
       'Москва, улица Богданова, 2к1, ш. Боровское (3 км до МКАД), ш. Киевское (8 км до МКАД)'],
      dtype=object)

Убираем Москву, т.к. весь датасет по Москве.

In [17]:
df['Адрес'] = df['Адрес'].map(lambda x: x[8:])

### Площадь

In [18]:
df['Площадь, м2'].unique()

array(['200.0/20.0', '198.0/95.0/18.0', '200.0/116.0/4.0', ..., '43.1',
       '52.5/10.0', '90.0/48.2/15.5'], dtype=object)

Нужно убрать лишние данные, оставить только общую площадь, преобразовать в тип данных float

In [19]:
df['Площадь, м2'] = df['Площадь, м2'].map(lambda x: float(x.split('/')[0]))

### Дом

In [20]:
df['Дом'].unique()

array(['5/16, Монолитный', '5/16, Монолитно-кирпичный', '5/16', ...,
       '12/23, Панельный', '12/25, Блочный', '2/2, Блочный'], dtype=object)

In [21]:
df['Этаж'] = df['Дом'].map(lambda x: int((x.split(', ')[0]).split('/')[0]))
df['Этажность'] = df['Дом'].map(lambda x: int((x.split(', ')[0]).split('/')[1]))
df['Тип дома'] = df['Дом'].map(lambda x: x.split(', ')[-1] if len(x.split(', '))> 1 else 'н/д')

In [22]:
df = df.drop('Дом', axis=1)

### Парковка

In [23]:
df['Парковка'].unique()

array(['подземная', nan, 'наземная', 'открытая', 'многоуровневая',
       'на крыше'], dtype=object)

Заполним данные по пропускам - предполагаем, что там нет парковки. Варианта "нет" нет в значениях, значит скорее всего это он.

In [24]:
df['Парковка'] = df['Парковка'].fillna('нет')

### Цена

In [25]:
df['Цена'].unique()

array(['500000.0 руб./ За месяц, Залог - 500000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес',
       '500000.0 руб./ За месяц, Залог - 500000 руб., Срок аренды - Длительный, Предоплата 1 мес',
       '400000.0 руб./ За месяц, Залог - 400000 руб., Срок аренды - Длительный, Предоплата 1 мес',
       ...,
       '25000.0 руб./ За месяц, Залог - 12000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 1 мес',
       '48000.0 руб./ За месяц, Залог - 40000 руб., Коммунальные услуги включены, Срок аренды - На несколько месяцев, Предоплата 1 мес',
       '55000.0 руб./ За месяц, Залог - 50000 руб., Коммунальные услуги включены, Срок аренды - Длительный, Предоплата 2 мес'],
      dtype=object)

In [26]:
df['Цена'] = df['Цена'].map(lambda x: float(x.split()[0]))

Поскольку поле цены заполнено неоднородно и целевой признак именно цена, оставим только его

### Телефоны

In [27]:
df = df.drop('Телефоны', axis=1)

Номера телефонов не будут влиять на качество модели. К удалению.

### Описание

Поскольку наша модель не будет работать на основании сложной обработки текста, удаляем этот столбец.

In [28]:
df = df.drop('Описание', axis=1)

### Ремонт

In [29]:
df['Ремонт'].unique()

array(['Дизайнерский', 'Евроремонт', 'Косметический', nan, 'Без ремонта'],
      dtype=object)

Пропуски по ремонту могут быть разными по смыслу. Заменим на отсутствие данных (н/д). Гипотеза: ремонт может существенно влиять на цену

In [30]:
df['Ремонт'] = df['Ремонт'].fillna('н/д')

### Площадь комнат, м2

In [31]:
df['Площадь комнат, м2'].unique()

array([nan, '25 25 20 25', '14-42-20-19', ..., '22-16-30', '32+13,6-18,7',
       '26,2+15,5-22,0'], dtype=object)

Данные не могут быть использованы, поскольку имеют разный формат перечисления, требуют проверки на соответствие общей площади. Возможны ошибки в указании площади (0 м2, 2 м2), есть дробные части через запятую, есть перечисления через запятую.

In [32]:
df = df.drop('Площадь комнат, м2', axis=1)

## Переименование столбцов

In [33]:
df.columns

Index(['ID  объявления', 'Количество комнат', 'Адрес', 'Площадь, м2',
       'Парковка', 'Цена', 'Ремонт', 'метро', 'Время до метро',
       'Расчет времени пешком', 'Этаж', 'Этажность', 'Тип дома'],
      dtype='object')

In [34]:
df.rename(columns = {
                    'ID  объявления':'add_id', 
                    'Количество комнат':'rooms',
                    'Адрес': 'address',
                    'Площадь, м2': 'square_m2',
                    'Парковка': 'parking',
                    'Цена': 'price',
                    'Ремонт': 'repair',
                    'метро': 'metro',
                    'Время до метро': 'time_to_metro',
                    'Расчет времени пешком': 'time_on_foot',
                    'Этаж': 'flat_floor',
                    'Этажность': 'house_floors',
                    'Тип дома': 'house_type'
                    }, inplace = True)

## Объединение двух частей по столбцам из разных csv

In [35]:
df_2 = pd.read_csv('/home/user/ds_bootcamp/real_estate/data_p2.csv',  sep='\t')
df_2 = df_2.drop('Unnamed: 0', axis = 1)
df_2 = df_2.rename(columns = {'ad_ID': 'add_id'})
df_2.head(3)

,add_id,balcony,windows,bathroom,with_children/animals,name_of_the_RC,house_series,ceiling_height_m,elevator,garbage_chute,furniture_rooms,furniture_kitchen,the_internet,tv
0,271271157,0,н/д,н/д,"Можно с детьми, Можно с животными",Новый Арбат,н/д,3.0,"Пасс,Груз",Да,1,1,1,1
1,271634126,0,На улицу и двор,Совмещенный,Можно с детьми,Новый Арбат,н/д,3.5,"Пасс,Груз",Нет,1,1,1,1
2,271173086,0,На улицу и двор,Совмещенный,Можно с детьми,Новый Арбат,н/д,3.2,Пасс,н/д,1,1,1,1


In [36]:
df_result = pd.merge(df, df_2, on='add_id', how='inner')
df_result.head(3)

,add_id,rooms,address,square_m2,parking,price,repair,metro,time_to_metro,time_on_foot,...,with_children/animals,name_of_the_RC,house_series,ceiling_height_m,elevator,garbage_chute,furniture_rooms,furniture_kitchen,the_internet,tv
0,271271157,4,"улица Новый Арбат, 27",200.0,подземная,500000.0,Дизайнерский,Смоленская,9,True,...,"Можно с детьми, Можно с животными",Новый Арбат,н/д,3.0,"Пасс,Груз",Да,1,1,1,1
1,271634126,4,"улица Новый Арбат, 27",198.0,подземная,500000.0,Дизайнерский,Смоленская,8,True,...,Можно с детьми,Новый Арбат,н/д,3.5,"Пасс,Груз",Нет,1,1,1,1
2,271173086,4,"улица Новый Арбат, 27",200.0,подземная,500000.0,Евроремонт,Смоленская,7,True,...,Можно с детьми,Новый Арбат,н/д,3.2,Пасс,н/д,1,1,1,1


## Создаем новый объединенный файл

In [37]:
# df_result.to_csv('data.csv')
df_result = pd.read_csv('/home/user/ds_bootcamp/real_estate/data.csv')
df_result = df_result

## Категоризация данных

Для удобства работы модели категоризируем все данные и перенесем их в новый файл data.csv

In [38]:
df_result = df_result.drop('Unnamed: 0', axis=1)

In [39]:
# Создаем экземпляр LabelEncoder

label_encoders = {}

# Находим все текстовые столбцы
text_columns = df_result.select_dtypes(include=['object']).columns

# Кодируем текстовые столбцы
for column in text_columns:
    le = LabelEncoder()
    df_result[column] = le.fit_transform(df_result[column])
    label_encoders[column] = le  # Сохраняем кодировщик для возможного обратного преобразования

df_result.head()

,add_id,rooms,address,square_m2,parking,price,repair,metro,time_to_metro,time_on_foot,...,with_children/animals,name_of_the_RC,house_series,ceiling_height_m,elevator,garbage_chute,furniture_rooms,furniture_kitchen,the_internet,tv
0,271271157,4,10682,200.0,5,500000.0,1,237,9,True,...,1,760,363,3.0,2,0,1,1,1,1
1,271634126,4,10682,198.0,5,500000.0,1,237,8,True,...,0,759,363,3.5,2,1,1,1,1,1
2,271173086,4,10682,200.0,5,500000.0,2,237,7,True,...,0,759,363,3.2,1,2,1,1,1,1
3,272197456,4,6925,170.0,5,400000.0,2,237,3,True,...,2,1083,363,3.2,1,2,1,1,1,1
4,273614615,2,10679,58.0,3,225000.0,2,12,7,True,...,3,147,363,3.9,2,0,1,1,1,1


In [40]:
df_result.to_csv('data_cat.csv', index=False)
df_cat = pd.read_csv('/home/user/ds_bootcamp/real_estate/data_cat.csv')
df_cat

,add_id,rooms,address,square_m2,parking,price,repair,metro,time_to_metro,time_on_foot,...,with_children/animals,name_of_the_RC,house_series,ceiling_height_m,elevator,garbage_chute,furniture_rooms,furniture_kitchen,the_internet,tv
0,271271157,4,10682,200.0,5,500000.0,1,237,9,True,...,1,760,363,3.00,2,0,1,1,1,1
1,271634126,4,10682,198.0,5,500000.0,1,237,8,True,...,0,759,363,3.50,2,1,1,1,1,1
2,271173086,4,10682,200.0,5,500000.0,2,237,7,True,...,0,759,363,3.20,1,2,1,1,1,1
3,272197456,4,6925,170.0,5,400000.0,2,237,3,True,...,2,1083,363,3.20,1,2,1,1,1,1
4,273614615,2,10679,58.0,3,225000.0,2,12,7,True,...,3,147,363,3.90,2,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20188,202336163,1,5607,32.0,3,35000.0,3,241,10,True,...,1,1083,363,2.64,1,0,1,1,1,1
20189,274654844,1,4868,38.7,3,45000.0,2,241,7,True,...,3,611,363,2.98,2,2,1,1,0,1
20190,268679909,2,1310,43.1,3,50000.0,1,241,6,True,...,0,1083,363,2.98,3,2,0,1,1,0
20191,274807525,2,9176,52.5,2,55000.0,2,241,11,True,...,3,1083,363,2.65,1,0,1,1,0,0
